# Piecewise Effects

Model **non-linear investment costs** with economies of scale and discrete size tiers.

This notebook demonstrates:
- **PiecewiseEffects**: Non-linear cost functions for investments
- **Gaps between pieces**: Representing discrete size tiers (unavailable sizes)
- How the optimizer selects from available size options

In [1]:
import numpy as np
import pandas as pd

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## The Problem: Discrete Size Tiers

Real equipment often comes in **discrete sizes** with gaps between options:

| Tier | Size Range | Cost per kWh | Notes |
|------|------------|--------------|-------|
| Small | 50-100 kWh | 0.20 €/kWh | Residential units |
| *Gap* | 100-200 kWh | *unavailable* | No products in this range |
| Medium | 200-400 kWh | 0.12 €/kWh | Commercial units |
| *Gap* | 400-500 kWh | *unavailable* | No products in this range |
| Large | 500-800 kWh | 0.06 €/kWh | Industrial units |

The gaps represent size ranges where no products are available from manufacturers.

## Define the Cost Curve with Gaps

Each piece defines a size tier. Gaps between pieces are **forbidden** zones.

In [2]:
# Piecewise costs with gaps between tiers
# Cost values are CUMULATIVE at each breakpoint
piecewise_costs = fx.PiecewiseEffects(
    piecewise_origin=fx.Piecewise(
        [
            fx.Piece(start=50, end=100),  # Small tier: 50-100 kWh
            fx.Piece(start=200, end=400),  # Medium tier: 200-400 kWh (gap: 100-200)
            fx.Piece(start=500, end=800),  # Large tier: 500-800 kWh (gap: 400-500)
        ]
    ),
    piecewise_shares={
        'costs': fx.Piecewise(
            [
                fx.Piece(start=10, end=20),  # 50kWh=10€, 100kWh=20€ → 0.20 €/kWh
                fx.Piece(start=24, end=48),  # 200kWh=24€, 400kWh=48€ → 0.12 €/kWh
                fx.Piece(start=30, end=48),  # 500kWh=30€, 800kWh=48€ → 0.06 €/kWh
            ]
        )
    },
)

print('Available size tiers:')
print('  Small:  50-100 kWh at 0.20 €/kWh')
print('  Medium: 200-400 kWh at 0.12 €/kWh')
print('  Large:  500-800 kWh at 0.06 €/kWh')

Available size tiers:
  Small:  50-100 kWh at 0.20 €/kWh
  Medium: 200-400 kWh at 0.12 €/kWh
  Large:  500-800 kWh at 0.06 €/kWh


In [3]:
timesteps = pd.date_range('2024-01-01', periods=24, freq='h')

# Electricity price: cheap at night, expensive during day
elec_price = np.array(
    [
        0.05,
        0.05,
        0.05,
        0.05,
        0.05,
        0.05,  # 00-06: night (cheap)
        0.15,
        0.20,
        0.25,
        0.25,
        0.20,
        0.15,  # 06-12: morning
        0.15,
        0.20,
        0.25,
        0.30,
        0.30,
        0.25,  # 12-18: afternoon (expensive)
        0.20,
        0.15,
        0.10,
        0.08,
        0.06,
        0.05,  # 18-24: evening
    ]
)

demand = np.full(24, 100)  # 100 kW constant demand

## Simple Arbitrage Scenario

A battery arbitrages between cheap night and expensive day electricity.

## Build and Solve the Model

In [4]:
fs = fx.FlowSystem(timesteps)

fs.add_elements(
    fx.Bus('Elec'),
    fx.Effect('costs', '€', is_standard=True, is_objective=True),
    # Grid with time-varying price
    fx.Source('Grid', outputs=[fx.Flow('Elec', bus='Elec', size=500, effects_per_flow_hour=elec_price)]),
    # Battery with PIECEWISE investment cost (discrete tiers)
    fx.Storage(
        'Battery',
        charging=fx.Flow('charge', bus='Elec', size=fx.InvestParameters(maximum_size=400)),
        discharging=fx.Flow('discharge', bus='Elec', size=fx.InvestParameters(maximum_size=400)),
        capacity_in_flow_hours=fx.InvestParameters(
            piecewise_effects_of_investment=piecewise_costs,
            minimum_size=0,
            maximum_size=800,
        ),
        eta_charge=0.95,
        eta_discharge=0.95,
        initial_charge_state=0,
    ),
    fx.Sink('Demand', inputs=[fx.Flow('Elec', bus='Elec', size=1, fixed_relative_profile=demand)]),
)

fs.optimize(fx.solvers.HighsSolver());

## Visualize the Cost Curve

The
plot
shows
the
three
discrete
tiers
with gaps between them.

In [5]:
piecewise_costs.plot(title='Battery Investment Cost (Discrete Tiers)')

PlotResult(data=<xarray.Dataset> Size: 280B
Dimensions:  (trace: 3, point: 2)
Coordinates:
  * point    (point) <U5 40B 'start' 'end'
Dimensions without coordinates: trace
Data variables:
    origin   (trace, point) int64 48B 50 100 200 400 500 800
    share    (trace, point) int64 48B 10 20 24 48 30 48
    effect   (trace) <U5 60B 'costs' 'costs' 'costs'
    piece    (trace) <U7 84B 'Piece 0' 'Piece 1' 'Piece 2', figure=Figure({
    'data': [{'hovertemplate': 'piece=Piece 0<br>line_id=0<br>origin=%{x}<br>share=%{y}<extra></extra>',
              'legendgroup': 'Piece 0',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Piece 0',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'MmQ=', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': 'ChQ=', 'dtype': 'i1'},
              'yaxis': 'y'},
             {'hovertemplate': 'piece=Piece 1<br>line_id=1<br>origin=%{x}<br>share=%{y}<extra></extra>',
              'legendgroup': 'Piece 1',
              'line': {'color': '#636EFA', 'dash': 'dot'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Piece 1',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'yACQAQ==', 'dtype': 'i2'},
              'xaxis': 'x',
              'y': {'bdata': 'GDA=', 'dtype': 'i1'},
              'yaxis': 'y'},
             {'hovertemplate': 'piece=Piece 2<br>line_id=2<br>origin=%{x}<br>share=%{y}<extra></extra>',
              'legendgroup': 'Piece 2',
              'line': {'color': '#636EFA', 'dash': 'dash'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Piece 2',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': '9AEgAw==', 'dtype': 'i2'},
              'xaxis': 'x',
              'y': {'bdata': 'HjA=', 'dtype': 'i1'},
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'piece'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Battery Investment Cost (Discrete Tiers)'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Origin'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': ''}}}
}))

## Results: Which Tier Was Selected?

In [6]:
battery_size = fs.solution['Battery|size'].item()
total_cost = fs.solution['costs'].item()

# Determine which tier was selected
if battery_size < 1:
    tier = 'None'
elif battery_size <= 100:
    tier = 'Small (50-100 kWh)'
elif battery_size <= 400:
    tier = 'Medium (200-400 kWh)'
else:
    tier = 'Large (500-800 kWh)'

print(f'Selected tier: {tier}')
print(f'Battery size:  {battery_size:.0f} kWh')
print(f'Total cost:    {total_cost:.1f} €')

Selected tier: Large (500-800 kWh)
Battery size:  800 kWh
Total cost:    249.0 €


## Storage Operation

In [7]:
fs.statistics.plot.balance('Elec')

PlotResult(data=<xarray.Dataset> Size: 1kB
Dimensions:             (time: 25)
Coordinates:
  * time                (time) datetime64[ns] 200B 2024-01-01 ... 2024-01-02
Data variables:
    Grid(Elec)          (time) float64 200B -100.0 -100.0 -100.0 ... -100.0 nan
    Battery(discharge)  (time) float64 200B -0.0 1.243e-13 ... 1.243e-13 nan
    Battery(charge)     (time) float64 200B 0.0 -1.385e-13 ... -1.385e-13 nan
    Demand(Elec)        (time) float64 200B 100.0 100.0 100.0 ... 100.0 nan, figure=Figure({
    'data': [{'hovertemplate': 'variable=Grid(Elec)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Grid(Elec)',
              'marker': {'color': '#636EFA', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Grid(Elec)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2024-01-01T00:00:00.000000000', '2024-01-01T01:00:00.000000000',
                          '2024-01-01T02:00:00.000000000', '2024-01-01T03:00:00.000000000',
                          '2024-01-01T04:00:00.000000000', '2024-01-01T05:00:00.000000000',
                          '2024-01-01T06:00:00.000000000', '2024-01-01T07:00:00.000000000',
                          '2024-01-01T08:00:00.000000000', '2024-01-01T09:00:00.000000000',
                          '2024-01-01T10:00:00.000000000', '2024-01-01T11:00:00.000000000',
                          '2024-01-01T12:00:00.000000000', '2024-01-01T13:00:00.000000000',
                          '2024-01-01T14:00:00.000000000', '2024-01-01T15:00:00.000000000',
                          '2024-01-01T16:00:00.000000000', '2024-01-01T17:00:00.000000000',
                          '2024-01-01T18:00:00.000000000', '2024-01-01T19:00:00.000000000',
                          '2024-01-01T20:00:00.000000000', '2024-01-01T21:00:00.000000000',
                          '2024-01-01T22:00:00.000000000', '2024-01-01T23:00:00.000000000',
                          '2024-01-02T00:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAWcD///////9YwAgAAAAAAF' ... '////9YwP///////1jAAAAAAAAA+P8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Battery(discharge)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Battery(discharge)',
              'marker': {'color': '#EF553B', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Battery(discharge)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2024-01-01T00:00:00.000000000', '2024-01-01T01:00:00.000000000',
                          '2024-01-01T02:00:00.000000000', '2024-01-01T03:00:00.000000000',
                          '2024-01-01T04:00:00.000000000', '2024-01-01T05:00:00.000000000',
                          '2024-01-01T06:00:00.000000000', '2024-01-01T07:00:00.000000000',
                          '2024-01-01T08:00:00.000000000', '2024-01-01T09:00:00.000000000',
                          '2024-01-01T10:00:00.000000000', '2024-01-01T11:00:00.000000000',
                          '2024-01-01T12:00:00.000000000', '2024-01-01T13:00:00.000000000',
                          '2024-01-01T14:00:00.000000000', '2024-01-01T15:00:00.000000000',
                          '2024-01-01T16:00:00.000000000', '2024-01-01T17:00:00.000000000',
                          '2024-01-01T18:00:00.000000000', '2024-01-01T19:00:00.000000000',
                          '2024-01-01T20:00:00.000000000', '2024-01-01T21:00:00.000000000',
                          '2024-01-01T22:00:00.000000000', '2024-01-01T23:00:00.000000000',
                          '2024-01-02T00:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAAIDMt3zLt3xBPanl

## Best Practice: PiecewiseEffects with Gaps

```python
fx.PiecewiseEffects(
    piecewise_origin=fx.Piecewise([
        fx.Piece(start=50, end=100),   # Tier 1
        fx.Piece(start=200, end=400),  # Tier 2 (gap: 100-200 forbidden)
    ]),
    piecewise_shares={
        'costs': fx.Piecewise([
            fx.Piece(start=10, end=20),  # Cumulative cost at tier 1 boundaries
            fx.Piece(start=24, end=48),  # Cumulative cost at tier 2 boundaries
        ])
    },
)
```

**Key points:**
- Gaps between pieces = forbidden size ranges
- Cost values are **cumulative** at each boundary
- Use when equipment comes in discrete tiers

## Previous: Piecewise Conversion

See **[06b-piecewise-conversion](06b-piecewise-conversion.ipynb)** for modeling minimum load constraints with `PiecewiseConversion` + `StatusParameters`.